<!--
tags: ["sequential chats"]
description: |
    Use AutoGen to solve a set of tasks with a sequence of chats.
-->

<!--
tags: ["sequential chat"]
description: |
    Use AutoGen.initiate_chats to solve a set of tasks with a sequence of chats.
-->
# Solving Multiple Tasks in a Sequence of Chats

This notebook showcases how to use the new chat interface `autogen.initiate_chats` to solve a set of tasks with a sequence of chats. 

\:\:\:info Requirements

Install `pyautogen`:
```bash
pip install pyautogen
```

For more information, please refer to the [installation guide](/docs/installation/).

\:\:\:


In [ ]:
import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")
llm_config = {"config_list": config_list}

\:\:\:tip

Learn more about the various ways to configure LLM endpoints [here](/docs/llm_configuration).

\:\:\:

### Example Tasks
Below are three example tasks, with each task being a string of text describing the request. The completion of later tasks requires or benefits from the results of previous tasks.


In [ ]:
financial_tasks = [
    """What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?""",
    """Investigate possible reasons of the stock performance.""",
]

writing_tasks = ["""Develop an engaging blog post using any information provided."""]

### Solve tasks with a series of chats

The `autogen.initiate_chats` interface can take a list of dictionaries as inputs. Each dictionary preserves the following fields: 
- `sender`: a conversable agent as the sender;
- `recipient`: a conversable agent as the recipient;
- `message`: is a string of text (typically a message containing the task);
- `summary_method`: A string specifying the method to get a summary from the chat. Currently supported choices include `last_msg`, which takes the last message from the chat history as the summary, and `reflection_with_llm`, which uses an LLM call to reflect on the chat history and summarize a takeaway;
- `summary_prompt`: A string specifying how to instruct an LLM-backed agent (either the recipient or the sender in the chat) to reflect on the chat history and derive a summary. If not otherwise specified, a default prompt will be used when `summary_method` is `reflection_with_llm`.
"Summarize the takeaway from the conversation. Do not add any introductory phrases. If the intended request is NOT properly addressed, please point it out."
- `carryover`: A string or a list of string to specify additional context to be used in the chat. With `initiate_chats`, summary from previous chats will be added as carryover. They will be appended after the carryover provided by the user.

In [9]:
financial_assistant = autogen.AssistantAgent(
    name="Financial_assistant",
    llm_config=llm_config,
)
research_assistant = autogen.AssistantAgent(
    name="Researcher",
    llm_config=llm_config,
)
writer = autogen.AssistantAgent(
    name="writer",
    llm_config=llm_config,
    system_message="""
        You are a professional writer, known for
        your insightful and engaging articles.
        You transform complex concepts into compelling narratives.
        Reply "TERMINATE" in the end when everything is done.
        """,
)

user_proxy_auto = autogen.UserProxyAgent(
    name="User_Proxy_Auto",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)

user_proxy = autogen.UserProxyAgent(
    name="User_Proxy",
    human_input_mode="ALWAYS",  # ask human for input at each step
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "tasks",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
)


chat_results = autogen.initiate_chats(
    [
        {
            "sender": user_proxy_auto,
            "recipient": financial_assistant,
            "message": financial_tasks[0],
            "clear_history": True,
            "silent": False,
            "summary_method": "last_msg",
        },
        {
            "sender": user_proxy_auto,
            "recipient": research_assistant,
            "message": financial_tasks[1],
            "summary_method": "reflection_with_llm",
        },
        {
            "sender": user_proxy,
            "recipient": writer,
            "message": writing_tasks[0],
            "carryover": "I want to include a figure or a table of data in the blogpost.",
        },
    ]
)


********************************************************************************
Start a new chat with the following message: 
What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

With the following carryover: 


********************************************************************************
User_Proxy_Auto (to Financial_assistant):

What are the current stock prices of NVDA and TESLA, and how is the performance over the past month in terms of percentage change?

--------------------------------------------------------------------------------
Financial_assistant (to User_Proxy_Auto):

To get the current stock prices of NVDA (NVIDIA Corporation) and TSLA (Tesla, Inc.), along with their performance over the past month in terms of percentage change, we can use a Python library like `yfinance`. This library allows downloading of historical market data from Yahoo Finance.

Let's proceed in the following steps:


#### Check chat results
The `initiate_chat` method returns a `ChatResult` object, which is a dataclass object storing information about the chat. Currently, it includes the following attributes:

- `chat_history`: a list of chat history.
- `summary`: a string of chat summary. A summary is only available if a summary_method is provided when initiating the chat.
- `cost`: a tuple of (total_cost, total_actual_cost), where total_cost is a dictionary of cost information, and total_actual_cost is a dictionary of information on the actual incurred cost with cache.
- `human_input`: a list of strings of human inputs solicited during the chat. (Note that since we are setting `human_input_mode` to `NEVER` in this notebook, this list is always empty.)

In [11]:
for i, chat_res in enumerate(chat_results):
    print(f"*****{i}th chat*******:")
    print(chat_res.summary)
    print("Human input in the middle:", chat_res.human_input)
    print("Conversation cost: ", chat_res.cost)
    print("\n\n")

*****0th chat*******:
The script has executed successfully and provided the following results:

- The current stock price for NVIDIA Corporation (NVDA) is $722.48.
- The current stock price for Tesla, Inc. (TSLA) is $188.13.
- NVDA has experienced a percentage increase of 28.14% over the past month.
- TSLA has experienced a percentage decrease of -14.45% over the past month.

These results indicate that over the past month, NVIDIA’s stock has seen a significant increase in value, whereas Tesla’s stock has decreased in value. Keep in mind that stock prices and their performance are subject to rapid change and can vary within minutes.

If you require any further assistance, please let me know. Otherwise, that concludes our task.


Human input in the middle: []
Conversation cost:  ({'total_cost': 0.08859, 'gpt-4': {'cost': 0.08859, 'prompt_tokens': 1597, 'completion_tokens': 678, 'total_tokens': 2275}}, {'total_cost': 0})



*****1th chat*******:
To investigate the stock performance of NV